<img src="./images/DLI_Header.png">

# Scaling to Multiple GPUs with Horovod

[Horovod](https://github.com/horovod/horovod) is a distributed deep learning training framework. It is available for TensorFlow, Keras, PyTorch, and Apache MXNet. In this lab you will learn about what Horovod is and how to use it, by distributing across multiple GPUs the training of the classification model we started with in Exercise 3 of Lab 1.

## Lab Outline

The progression of this lab is as follows:

- A high level introduction to Horovod, including its ties to the parallel computing protocol MPI, and the additional details that must be taken into account when using a parallel computing framework like Horovod.
- An overview and initial run of the existing code base that you will be refactoring with Horovod, which is a classification model using Keras and the Fashion-MNIST dataset, currently built to run on a single GPU.
- A multi-step refactor of the existing code base so that it uses Horovod to run distributed across this environment's available GPUs, introducing Horovod concepts and techniques throughout.
- A final run of the refactored and distributed code base, with discussion of its speed up.

This lab draws heavily on content provided in the [Horovod tutorials](https://github.com/horovod/tutorials).

## Learning Objectives

By the time you complete this lab you will be able to:

- Discuss what Horovod is, how it works, and why it is an effective tool for distributed training.
- Use Horovod to refactor or build deep learning models that train distributed across multiple GPUs.

## Introduction to Horovod

[Horovod](https://github.com/horovod/horovod) is an open source tool originally [developed by Uber](https://eng.uber.com/horovod/) to support their need for faster deep learning model training across their many engineering teams. It is part of a growing ecosystem of approaches to distributed training, including for example [Distributed TensorFlow](https://www.tensorflow.org/deploy/distributed). Uber decided to develop a solution that utilized [MPI](https://en.wikipedia.org/wiki/Message_Passing_Interface) for distributed process communication, and the [NVIDIA Collective Communications Library (NCCL)](https://developer.nvidia.com/nccl) for its highly optimized implementation of reductions across distributed processes and nodes. The resulting Horovod package delivers on its promise to scale deep learning model training across multiple GPUs and multiple nodes, with only minor code modification and intuitive debugging.

Since its inception in 2017 Horovod has matured significantly, extending its support from just TensorFlow to Keras, PyTorch, and Apache MXNet. Horovod is extensively tested and has been used on some of the largest DL training runs done to date, for example, supporting **exascale** deep learning on the [Summit system, scaling to over **27,000 V100 GPUs**](https://arxiv.org/pdf/1810.01993.pdf):

![horovod scaling](./images/horovod_scaling.jpg)

Let's import Horovod now so that we can query it later on. The convention is to import it as `hvd`.

In [1]:
import horovod.tensorflow.keras as hvd

### Horovod's MPI Roots

Horovod's connection to MPI runs deep, and for programmers familiar with MPI programming, much of what you program to distribute model training with Horovod will feel very familiar. For those unfamiliar with MPI programming, a brief discussion of some of the conventions and considerations required when distributing processes with Horovod, or MPI, is worthwhile.

Horovod, as with MPI, strictly follows the [Single-Program Multiple-Data (SPMD) paradigm](https://en.wikipedia.org/wiki/SPMD) where we implement the instruction flow of multiple processes in the same file/program. Because multiple processes are executing code in parallel, we have to take care about [race conditions](https://en.wikipedia.org/wiki/Race_condition) and also the synchronization of participating processes.

Horovod assigns a unique numerical ID or **rank** (an MPI concept) to each process executing the program. This rank can be accessed programmatically. As you will see below when writing Horovod code, by identifying a process's rank programmatically in the code we can take steps such as:

- Pin that process to its own exclusive GPU.
- Utilize a single rank for broadcasting values that need to be used uniformly by all ranks.
- Utilize a single rank for collecting and/or reducing values produced by all ranks.
- Utilize a single rank for logging or writing to disk.

As you work through this course, keep these concepts in mind and especially that Horovod will be sending your single program to be executed in parallel by multiple processes. Keeping this in mind will support your intuition and understanding about why we do what we do with Horovod, even though you will only be making edits to a single program.

## Overview of Existing Model Files

On the left hand side of this lab environment, you will see a file directory with this notebook, a couple of Python files and a `solutions` directory.

The file `fashion_mnist.py` contains the Keras model that does not have any Horovod code while `solutions/fashion_mnist_solution.py` has all the Horovod features added. In this tutorial, we will guide you to transform `fashion_mnist.py` into `solutions/fashion_mnist_solution.py` step-by-step. As you work through exercises to complete this task, you can, if needed compare your code with the `solutions/fashion_mnist_after_step_N.py` files that correspond to the step you are at.

## Baseline: train the model

Before we go into modifications required to scale our WideResNet model, please make sure you can train the single GPU version of the model. This is the same as where we left off at the end of Lab 1, except that now we are using the full dataset. We'll just run a few epochs with a relatively large batch size. This will take a bit longer than before, so feel free to read ahead while this is training. Take note of how long the training took when it is done.

In [2]:
!python fashion_mnist.py --epochs 5 --batch-size 512

2022-07-26 11:41:53.548469: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-07-26 11:41:54.944887: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-26 11:41:54.945868: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-07-26 11:41:55.108445: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1038] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-26 11:41:55.109717: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1746] Found device 0 with properties: 
pciBusID: 0000:00:1b.0 name: Tesla V100-SXM2-16GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 15.78GiB deviceMemoryBandwidth: 836.37GiB/s
2022-07-26 11:41:55.109846: I tensorflow/stream_executor/cuda/cuda_gpu

## Modify the training script

We are going to start making modifications to the training script. Before we do, let's make a copy of it on disk -- that way, if you make a mistake and want to back up to the beginning, you have a reference copy to refer to.

In [3]:
!cp fashion_mnist.py fashion_mnist_original.py

Double-click `fashion_mnist.py` in the left pane to open it in the editor.

### 1. Initialize Horovod and select the GPU to run on

Naturally we need to start by importing Horovod. 

**Exercise**: Add `import horovod.tensorflow.keras as hvd` to the training script and initialize Horovod before the argument parsing:

```python
# Horovod: initialize Horovod.
hvd.init()
```

(look for the `TODO: Step 1` lines).

With Horovod, which can run multiple processes across multiple GPUs, you typically use a single GPU per training process. Part of what makes Horovod simple to use is that it utilizes MPI, and as such, uses much of the MPI nomenclature. The concept of a **rank** in MPI is of a unique process ID. In this lab we will be using the term "rank" extensively. If you would like to know more about MPI concepts that are utilized heavily in Horovod, please refer to [the Horovod documentation](https://github.com/horovod/horovod/blob/master/docs/concepts.rst).

Schematically, let's look at how MPI can run multiple GPU processes across multiple nodes. Note how each process, or rank, is pinned to a specific GPU:

<center>
<img src="https://user-images.githubusercontent.com/16640218/53518255-7d5fc300-3a85-11e9-8bf3-5d0e8913c14f.png" width="400"></img>
</center>

With this method we do not have to deal with placing specific data on specific GPUs. Instead, you just specify which GPU you would like to use in the beginning of your script. 

Before we get there, let's refresh ourselves on how to work with multiple GPUs on a node. On the NVIDIA platform, CUDA, if we have N GPUs they are uniquely numbered from 0 to N-1. In this lab we won't worry about how the numbering is selected or whether the order matters. If we want to pin our process to use a specific GPU, say GPU 1, we can use the following TensorFlow code:

```python
# Pin to GPU 1
gpu_id = 1
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[gpu_id], True)
tf.config.experimental.set_visible_devices(gpus[gpu_id], 'GPU')
```

The `gpus` list contains all of the GPUs that this TensorFlow process is allowed by CUDA to see, from which it will select one to use by default. If you want to use multiple GPUs from this list, you can do so by manually controlling which "device" to use (see TensorFlow's [documentation](https://www.tensorflow.org/guide/gpu) for more information). Typically one would do so when manual control over the distribution of data is needed, and a common use case is model parallelism (which Horovod [does not natively support](https://github.com/horovod/horovod/issues/96)). In this lab we will only be using one GPU per rank. (Note: the `set_memory_growth()` call, which tells TensorFlow to allocate only the memory that it needs instead of pre-allocating the full GPU memory, is not strictly related to Horovod, but is often recommended.)

Now let's test your understanding of how this works. First, let's identify how many GPUs are on our node:

In [4]:
!nvidia-smi

Tue Jul 26 11:48:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:1B.0 Off |                    0 |
| N/A   33C    P0    35W / 300W |      0MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:00:1C.0 Off |                    0 |
| N/A   31C    P0    35W / 300W |      0MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

The "!" prefix means that we execute the above in the terminal; now let's do this in actual terminal. Open a new launcher (File > New Launcher in the menu bar), select the "Terminal" option, execute the `nvidia-smi` command there, and verify it provides the same output. Notice that there is a "GPU-Util" column, which measures the GPU's utilization. It tells you what fraction of the last second the GPU was in use. We can thus easily monitor GPU activity by regularly checking this output. One way to do that is using the Linux utility [watch](https://en.wikipedia.org/wiki/Watch_(Unix)): `watch -n 5 nvidia-smi` will set up a loop that refreshes the `nvidia-smi` output every 5 seconds. Make sure you run that in the separate terminal window, not here in the notebook, because the notebook can only run one process at a time. You can type Ctrl+C in the terminal to end the loop later.

**Exercise**: set up `nvidia-smi` to regularly monitor the GPU activity in a terminal as above, and then here in the notebook start a training process. Then switch back to the terminal and watch the GPU activity. Can you verify that only one GPU is used? Does it match the GPU ID you asked for in the training script? Also, keep an eye out for other utilization metrics like power consumption and memory usage.

In [9]:
!mpirun -np 1 python fashion_mnist.py --epochs 1 --batch-size 512

2022-07-26 11:54:29.622272: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-07-26 11:54:30.994295: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-26 11:54:30.995202: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-07-26 11:54:31.170310: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1038] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-26 11:54:31.171562: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1746] Found device 0 with properties: 
pciBusID: 0000:00:1b.0 name: Tesla V100-SXM2-16GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 15.78GiB deviceMemoryBandwidth: 836.37GiB/s
2022-07-26 11:54:31.171649: I tensorflow/stream_executor/cuda/cuda_gpu

Now let's modify the above code such that Horovod can automatically do the right thing for any number of training processes. Programmatically, we can arbitrarily select the GPU that corresponds to the Horovod rank and use that one. Since we might be using multiple nodes, and the Horovod rank is a unique identifier across all ranks in the training process, we want to identify our rank locally on the node, which is provided by the `local_rank` specifier:

```python
# Horovod: pin GPU to be used to process local rank (one GPU per process)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    tf.config.experimental.set_memory_growth(gpus[hvd.local_rank()], True)
    tf.config.experimental.set_visible_devices(gpus[hvd.local_rank()], 'GPU')
```

Check out the documentation for both functions:

In [6]:
?hvd.rank

Signature: hvd.rank()
Docstring:
A function that returns the Horovod rank of the calling process.

Returns:
  An integer scalar with the Horovod rank of the calling process.
File:      /usr/local/lib/python3.8/dist-packages/horovod/common/basics.py
Type:      method


In [7]:
?hvd.local_rank

Signature: hvd.local_rank()
Docstring:
A function that returns the local Horovod rank of the calling process, within the
node that it is running on. For example, if there are seven processes running
on a node, their local ranks will be zero through six, inclusive.

Returns:
  An integer scalar with the local Horovod rank of the calling process.
File:      /usr/local/lib/python3.8/dist-packages/horovod/common/basics.py
Type:      method


**Exercise**: with this knowledge in hand, edit `fashion_mnist.py` to pin one GPU to each rank using its local rank ID, immediately after where you have already initialized Horovod.

Again, look for `TODO: Step 1` lines in the code. If you get stuck, refer to `solutions/fashion_mnist_after_step_01.py`.

**Exercise**: Now let's test that you got this right too. Run the training script for just one epoch to make sure everything works. We'll get into the habit of launching with the MPI job launcher, `mpirun`, even though for a single process run this is unnecessary (see below for more details). Using `nvidia-smi` in the terminal, verify that only one training process is running, and note which GPU is being used. Is it the one you expect?

In [8]:
!mpirun -np 1 python fashion_mnist.py --epochs 1 --batch-size 512

2022-07-26 11:49:53.563525: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-07-26 11:49:54.980915: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-26 11:49:54.981870: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-07-26 11:49:55.153370: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1038] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-26 11:49:55.154653: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1746] Found device 0 with properties: 
pciBusID: 0000:00:1b.0 name: Tesla V100-SXM2-16GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 15.78GiB deviceMemoryBandwidth: 836.37GiB/s
2022-07-26 11:49:55.154742: I tensorflow/stream_executor/cuda/cuda_gpu

`mpirun` is a script that launches N copies of the training script, where N is the argument to `-np`. We'll be using it later to coordinate the training process. The processes are launched in the MPI environment, so they can communicate between each other through a standardized API that Horovod handles for us, though we haven't instructed the training script to actually coordinate yet; at present we will just launch N independent copies of the same training script. We can try this out now by running as many processes (ranks) as there are GPUs.

Watch the output and see if the training process appears to be coordinated -- does the training actually work? Do all of the processes run on the GPU you expect them to? Does the output of `nvidia-smi` show all GPUs being utilized?

In [12]:
num_gpus = 3

In [11]:
!mpirun -np $num_gpus python fashion_mnist.py --epochs 1 --batch-size 512

2022-07-26 11:55:38.351759: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-07-26 11:55:38.351763: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-07-26 11:55:38.351761: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-07-26 11:55:38.351765: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-07-26 11:55:39.842380: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-26 11:55:39.842818: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-26 11:55:39.842981: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-

### 2. Print verbose logs only on the first worker

You may have noticed that all N TensorFlow processes were simultaneously printing their progress to stdout (standard output) and overwriting the output from the other processes. This results in confusing output -- we only want to see the state of the output once at any given time. To accomplish this, we can arbitrarily select a single rank to display the training progress. By convention, we typically call rank 0 the "root" rank and use it for logistical work such as I/O when only one rank is required.

**Exercise**: Edit `fashion_mnist.py` so that you only set `verbose = 1` if it is the first worker (with rank equal to 0) executing the code.

Look for `TODO: Step 2` in `fashion_mnist.py`. If you get stuck, refer to `solutions/fashion_mnist_after_step_02.py`.

Re-run the training session to make sure that you now see the expected output. We'll run for 3 epochs this time for comparison to the next exercise. While it's running, you can start working on Step 3.

In [13]:
!mpirun -np $num_gpus python fashion_mnist.py --epochs 3 --batch-size 512

2022-07-26 11:56:45.571269: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-07-26 11:56:45.571289: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-07-26 11:56:45.571295: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-07-26 11:56:47.047124: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-26 11:56:47.047235: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-26 11:56:47.047278: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-26 11:56:47.048231: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-07-26 11

### 3. Add distributed optimizer

Horovod uses an operation that averages gradients across workers. Implementing this is very straightforward and just requires wrapping an existing optimizer (`tensorflow.keras.optimizers.Optimizer`) with a Horovod distributed optimizer (`horovod.tensorflow.keras.DistributedOptimizer`).

In [14]:
?hvd.DistributedOptimizer

Signature:
hvd.DistributedOptimizer(
    optimizer,
    name=None,
    device_dense='',
    device_sparse='',
    compression=<class 'horovod.tensorflow.compression.NoneCompressor'>,
    sparse_as_dense=False,
    gradient_predivide_factor=1.0,
    op=0,
    backward_passes_per_step=1,
    average_aggregated_gradients=False,
)
Docstring:
An optimizer that wraps another keras.optimizers.Optimizer, using an allreduce to
average gradient values before applying gradients to model weights.

Args:
    optimizer: Optimizer to use for computing gradients and applying updates.
    name: Optional name prefix for the operations created when applying
          gradients. Defaults to "Distributed" followed by the provided
          optimizer type.
    device_dense: Device to be used for dense tensors. Uses GPU by default
                  if Horovod was build with HOROVOD_GPU_OPERATIONS.
    device_sparse: Device to be used for sparse tensors. Uses GPU by default
                   if Horovod was b

**Exercise**: wrap the optimizer (`opt` in `fashion_mnist.py`) with a Horovod distributed optimizer.

Look for `TODO: Step 3` in `fashion_mnist.py`. If you get stuck, refer to `solutions/fashion_mnist_after_step_03.py`.

Re-run the training now and see if you get a reasonable answer. Is the accuracy any better? Note that we are only training for a few epochs, and the results will depend on the initial random weights, so do not draw any strong conclusions here.

In [15]:
!mpirun -np $num_gpus python fashion_mnist.py --epochs 3 --batch-size 512

2022-07-26 11:58:21.622070: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-07-26 11:58:21.622069: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-07-26 11:58:21.622070: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-07-26 11:58:23.030548: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-26 11:58:23.030577: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-26 11:58:23.030752: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-26 11:58:23.031465: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-07-26 11

### 4. Initialize random weights on only one processor

Data parallel stochastic gradient descent, at least in its traditionally defined sequential algorithm, requires weights to be synchronized between all processors. We already know that this is accomplished for backpropagation by averaging out the gradients among all processors prior to the weight updates. Then the only other required step is for the weights to be synchronized initially. Assuming we start from the beginning of the training (we won't handle checkpoint/restart in this lab, but [it is a straightforward extension](https://horovod.readthedocs.io/en/latest/keras.html)), this means that every processor needs to have the same random weights.


In a previous section, we mentioned that the first worker would broadcast parameters to the rest of the workers.  We will use `horovod.tensorflow.keras.callbacks.BroadcastGlobalVariablesCallback` to make this happen. Execute the following cell to get more information about the method:

In [16]:
?hvd.callbacks.BroadcastGlobalVariablesCallback

Init signature: hvd.callbacks.BroadcastGlobalVariablesCallback(root_rank, device='')
Docstring:     
Keras Callback that will broadcast all global variables from root rank
to all other processes during initialization.

This is necessary to ensure consistent initialization of all workers when
training is started with random weights or restored from a checkpoint.
Init docstring:
Construct a new BroadcastGlobalVariablesCallback that will broadcast all
global variables from root rank to all other processes during initialization.

Args:
    root_rank: Rank that will send data, other ranks will receive data.
    device: Device to be used for broadcasting. Uses GPU by default
            if Horovod was build with HOROVOD_GPU_OPERATIONS.
File:           /usr/local/lib/python3.8/dist-packages/horovod/tensorflow/keras/callbacks.py
Type:           type
Subclasses:     


**Exercise**: append this callback to our list of callbacks. Note the argument required for this callback, the rank of the root worker.

Look for `TODO: Step 4` in `fashion_mnist.py`. If you get stuck, refer to `solutions/fashion_mnist_after_step_04.py`.

Again, run the training session for a few epochs just to make sure things work, and notice if it affected the outcome.

In [17]:
!mpirun -np $num_gpus python fashion_mnist.py --epochs 3 --batch-size 512

2022-07-26 11:59:37.308767: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-07-26 11:59:37.308769: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-07-26 11:59:37.308767: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-07-26 11:59:38.828041: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-26 11:59:38.828473: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-26 11:59:38.828909: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-07-26 11:59:38.828990: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-26 11

### 5. Modify training loop to execute fewer steps per epoch

As it stands, we are running the same number of steps per epoch for the serial training implementation. But since we have increased the number of workers by a factor of N, that means we're doing N times more work (when we sum the amount of work done over all processes). Our target was to get the *same* answer in less time (that is, to speed up the training), so we want to keep the total amount of work done the same (that is, to process the same number of examples in the dataset). This means we need to do a factor of N *fewer* steps per epoch, so the number of steps goes to `steps_per_epoch / number_of_workers`.

We will also distribute the validation task, handling `3 * num_test_iterations / number_of_workers` steps on each worker. While we could just do `num_test_iterations / number_of_workers` on each worker to get a linear speedup in the validation, the multiplier **3** provides over-sampling of the validation data and helps to increase the probability that every validation example will be evaluated.

In [18]:
?hvd.size

Signature: hvd.size()
Docstring:
A function that returns the number of Horovod processes.

Returns:
  An integer scalar containing the number of Horovod processes.
File:      /usr/local/lib/python3.8/dist-packages/horovod/common/basics.py
Type:      method


**Exercise**: modify the `steps_per_epoch` and `validation_steps` arguments for `model.fit` to follow the plan just outlined. This environment uses Python 3, and each of these arguments expect integers, so take care to round any potential floating point values down to the nearest integer.

Look for `TODO: Step 5` in `fashion_mnist.py`. If you get stuck, refer to `solutions/fashion_mnist_after_step_05.py`.

In [19]:
!mpirun -np $num_gpus python fashion_mnist.py --epochs 3 --batch-size 512

2022-07-26 12:00:51.363996: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-07-26 12:00:51.364006: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-07-26 12:00:51.364020: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-07-26 12:00:52.871631: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-26 12:00:52.871801: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-26 12:00:52.871888: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-26 12:00:52.872518: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-07-26 12

### 6. Average validation results among workers

Since we are not validating the full dataset on each worker anymore, each worker will have different validation results. To improve validation metric quality and reduce variance, we will average validation results among all workers.

To do so, we can use `horovod.tensorflow.keras.callbacks.MetricAverageCallback`. Execute the following cell to get more information:

In [20]:
?hvd.callbacks.MetricAverageCallback

Init signature: hvd.callbacks.MetricAverageCallback(device='')
Docstring:     
Keras Callback that will average metrics across all processes at the
end of the epoch. Useful in conjuction with ReduceLROnPlateau,
TensorBoard and other metrics-based callbacks.

Note: This callback must be added to the callback list before the
ReduceLROnPlateau, TensorBoard or other metrics-based callbacks.
Init docstring:
Construct a new MetricAverageCallback that will average metrics
across all processes at the end of the epoch.

Args:
    device: Device to be used for allreduce. Uses GPU by default
            if Horovod was build with HOROVOD_GPU_OPERATIONS.
File:           /usr/local/lib/python3.8/dist-packages/horovod/tensorflow/keras/callbacks.py
Type:           type
Subclasses:     


**Exercise**: average the metrics among workers at the end of every epoch by injecting `MetricAverageCallback` after `BroadcastGlobalVariablesCallback`. Please note that this callback must be in the list before other metrics-based callbacks, `ReduceLROnPlateau`, `TensorBoard`, etc.

Look for `TODO: Step 6` in `fashion_mnist.py`. If you get stuck, refer to `solutions/fashion_mnist_after_step_06.py`.

In [21]:
!mpirun -np $num_gpus python fashion_mnist.py --epochs 3 --batch-size 512

2022-07-26 12:02:05.731704: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-07-26 12:02:05.731722: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-07-26 12:02:05.731729: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-07-26 12:02:07.218903: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-26 12:02:07.219438: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-26 12:02:07.219625: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-26 12:02:07.219790: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-07-26 12

### 7. CPU Comparison

Now that you've implemented the training on multiple GPUs, let's look at a comparison, training on just the CPU. You can do this by setting `CUDA_VISIBLE_DEVICES` to an empty string. You should see quite a difference!

In [22]:
!CUDA_VISIBLE_DEVICES= python fashion_mnist.py --epochs 1 --batch-size 512

2022-07-26 12:03:19.507296: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-07-26 12:03:20.952309: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-26 12:03:20.953247: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-07-26 12:03:20.975199: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-07-26 12:03:20.975240: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: eb84da8e3e31
2022-07-26 12:03:20.975252: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: eb84da8e3e31
2022-07-26 12:03:20.975328: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 460.27.4
2022-07-26 12:03:20.975370: I t

## Check your work

Congratulations!  If you made it this far, your `fashion_mnist.py` should now be fully distributed. To verify, compare `fashion_mnist.py` to `solutions/fashion_mnist_solution.py`.

<img src="./images/DLI_Header.png">